In [21]:
import torch
from torch import tensor
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from ast import literal_eval
from py_experimenter.experimenter import PyExperimenter
import torch
import mysql.connector
import openml


from py_experimenter.database_connector_mysql import DatabaseConnectorMYSQL

# def connect(self):

#     db = mysql.connector.connect(
#         host="db01-kiml.kiml.ifi.lmu.de",
#         user="jonas_h",
#         password="thiswillnotactuallybevalidforverylongsodontcountonit",
#         database="jonas_test",
#         ssl_disabled=False,
#     )
#     return db


# def _start_transaction(self, connection, readonly=False):
#     if not readonly:
#         connection.start_transaction()


# DatabaseConnectorMYSQL.connect = connect
# DatabaseConnectorMYSQL._start_transaction = _start_transaction

%matplotlib inline

In [22]:
experimenter = PyExperimenter(
    experiment_configuration_file_path="./experiments/config/cfg_simple_debug.yml",
)
exp_frame = experimenter.get_table()
# exp_frame = exp_frame[exp_frame.fraction_cal_samples >= 0.19999]
exp_frame = exp_frame[exp_frame.openml_id != 4534]
exp_frame = exp_frame[exp_frame.openml_id != 15]
exp_frame = exp_frame[exp_frame.openml_id != 31]


2025-01-07 10:21:52,162  | py-experimenter - INFO     | Found 4 keyfields
2025-01-07 10:21:52,163  | py-experimenter - INFO     | Found 2 resultfields
2025-01-07 10:21:52,163  | py-experimenter - INFO     | Found 1 logtables
2025-01-07 10:21:52,164  | py-experimenter - INFO     | Found logtable cp_rank_dev__new_cross__results
2025-01-07 10:21:52,164  | py-experimenter - WARNING  | No custom section defined in config
2025-01-07 10:21:52,164  | py-experimenter - WARNING  | No codecarbon section defined in config
2025-01-07 10:21:52,165  | py-experimenter - INFO     | Initialized and connected to database


In [23]:
import numpy as np
datasets = {}
num_classes = {}
dataset_renamer = {}
for id in exp_frame.openml_id.unique():
    dataset = openml.datasets.get_dataset(id.item())
    target_attribute = dataset.default_target_attribute
    X, y, _, _ = dataset.get_data(target=target_attribute)
    datasets[id] = dataset
    dataset_renamer[id] = dataset.name
    num_classes[id] = len(np.unique(y))

In [24]:
logtable = experimenter.get_logtable("results")
result_df = pd.merge(exp_frame, logtable, left_on="ID", right_on="experiment_id")

In [25]:
result_df.columns

Index(['ID_x', 'model', 'openml_id', 'master_seed', 'fraction_cal_samples',
       'creation_date', 'status', 'start_date', 'name', 'machine',
       'clf_seed_x', 'mccv_seed_x', 'end_date', 'error', 'ID_y',
       'experiment_id', 'timestamp', 'conformity_score', 'alpha', 'score_f1',
       'score_acc', 'score_bacc', 'coverage_mean', 'coverage_std',
       'efficiency_mean', 'efficiency_std', 'gradient_updates', 'clf_seed_y',
       'mccv_seed_y'],
      dtype='object')

In [26]:
df = result_df

result_df["dataset"] =  result_df["openml_id"].replace(dataset_renamer)
result_df["num_classes"] = result_df["openml_id"].replace(num_classes)

result_df["dataset"] = result_df["dataset"] + " (" + result_df["num_classes"].astype(str) + ")" 

cs_renamer = {"negative_ranker_skill":"Skill",
              "own_one_minus_phat": "Naive"}

result_df["conformity_score"] =  result_df["conformity_score"].replace(cs_renamer)

result_df["method"] = result_df["model"] + " + " + result_df["conformity_score"] 

# group_cols = ['dataset', 'alpha', 'fraction_cal_samples', 'model']
group_cols = ['dataset', 'alpha', 'method']
metrics_max = ['score_acc', 'coverage_mean']
metrics_min = ['efficiency_mean']
metrics = metrics_max + metrics_min

grouped_df = df.groupby(group_cols).agg({metric: 'mean' for metric in metrics}).reset_index()
agg_dict = {metric: 'max' for metric in metrics_max}
agg_dict.update({metric: 'min' for metric in metrics_min})
best_values = grouped_df.groupby(['dataset', 'alpha']).agg(agg_dict).reset_index()

In [27]:
def generate_latex_table_with_lines(df, group_cols, value_cols, best_values):
    """Generate a LaTeX table with multirow entries, bold the best values, and add lines between consecutive multirows."""
    
    def format_value(value, group, metric, max=True):
        """Format the value, making the highest value bold."""
        max_value = best_values.loc[
            (best_values['dataset'] == group['dataset']) & 
            (best_values['alpha'] == group['alpha']), metric].values[0]
            # (best_values['fraction_cal_samples'] == group['fraction_cal_samples']), metric].values[0]
        if value == max_value:
            return f"\\textbf{{{value:.4f}}}"
        else:
            return f"{value:.4f}"
    
    def recursive_build(df, group_cols):
        # if len(group_cols) == 1:
            # Base case: only one group left, print it directly
            latex_str = ""
            prev_row = None
            for row_id, row in df.iterrows():
                latex_str_tmp = ""
                col_id_list = []
                for col_id, col in enumerate(group_cols):
                    if prev_row is None or row[col] != prev_row[col]:
                        col_id_list.append(col_id)
                        filter_cols = group_cols[:col_id+1]
                        indices = []
                        values = []
                        for fcol in filter_cols:
                            indices.append(fcol)
                            values.append(row[fcol])
                        criteria = dict(zip(indices, values))
                        # Filter the DataFrame using query
                        mask = pd.Series([True] * len(df))

                        # Iterate over criteria to apply conditions
                        for key, value in criteria.items():
                            mask = mask & (df[key] == value)

                        # Filter the DataFrame
                        filtered_df = df[mask]

                        length = len(filtered_df)


                        latex_str_tmp += f" \\multirow{{{length}}}{{*}}{{{row[col]}}} & "
                        if col_id == len(group_cols) - 1:
                            latex_str_tmp += " & ".join([format_value(row[col], row, col) for col in value_cols]) + " \\\\\n"

                        # if prev_row is not None and col_id < len(group_cols) - 1:
                        # latex_str_tmp += " \\\\ \cline{%d-%d} \\\\" % (col_id + 1, len(group_cols) + len(value_cols))

                    else:
                        latex_str_tmp += " & "
                        if col_id == len(group_cols) - 1:
                            latex_str_tmp += " & ".join([format_value(row[col], row, col) for col in value_cols]) + " \\\\\n"
                if row_id > 0 and group_cols[min(col_id_list)] != "model":
                    latex_str += "\\cline{%d-%d}" % (min(col_id_list)+1, len(group_cols) + len(value_cols))
                latex_str += latex_str_tmp
                indices = []
                values = []
                prev_row = row
            return latex_str
        
    # Start recursive building
    latex_body = recursive_build(df, group_cols)

    # Complete LaTeX table
    num_columns = len(group_cols) + len(value_cols)
    col_format = 'l' * len(group_cols) + 'r' * len(value_cols)
    latex_table = f"""
\\begin{{tabular}}{{{col_format}}}
\\toprule
{' & '.join(group_cols)} & {' & '.join(value_cols)} \\\\
\\midrule
{latex_body}
\\bottomrule
\\end{{tabular}}
"""
    return latex_table

# Generate the LaTeX table with bold formatting and lines
latex_table = generate_latex_table_with_lines(grouped_df, group_cols, metrics, best_values)

# Print or save the LaTeX table
print(latex_table.replace("_", "\_"))



\begin{tabular}{lllrrr}
\toprule
dataset & alpha & method & score\_acc & coverage\_mean & efficiency\_mean \\
\midrule
 \multirow{21}{*}{dermatology (6)} &  \multirow{7}{*}{0.05} &  \multirow{1}{*}{classifier+APSConformityScore} & \textbf{0.9770} & 0.9959 & 1.0865 \\
\cline{3-6} &  &  \multirow{1}{*}{classifier+LACConformityScore} & \textbf{0.9770} & \textbf{0.9973} & 1.1270 \\
\cline{3-6} &  &  \multirow{1}{*}{classifier+Naive} & \textbf{0.9770} & 0.9703 & \textbf{0.9865} \\
\cline{3-6} &  &  \multirow{1}{*}{classifier+NaiveConformityScore} & \textbf{0.9770} & 0.9959 & 1.1230 \\
\cline{3-6} &  &  \multirow{1}{*}{classifier+RAPSConformityScore} & \textbf{0.9770} & 0.9959 & 1.1068 \\
\cline{3-6} &  &  \multirow{1}{*}{classifier+TopKConformityScore} & \textbf{0.9770} & 0.9959 & 1.1284 \\
\cline{3-6} &  &  \multirow{1}{*}{ranker+Skill} & 0.9743 & 0.9676 & 1.0027 \\
\cline{2-6} &  \multirow{7}{*}{0.1} &  \multirow{1}{*}{classifier+APSConformityScore} & \textbf{0.9770} & 0.9946 & 1.0662 \\

In [28]:
grouped_df

,dataset,alpha,method,score_acc,coverage_mean,efficiency_mean
0,dermatology (6),0.05,classifier+APSConformityScore,0.977027,0.995946,1.086486
1,dermatology (6),0.05,classifier+LACConformityScore,0.977027,0.997297,1.127027
2,dermatology (6),0.05,classifier+Naive,0.977027,0.970270,0.986486
3,dermatology (6),0.05,classifier+NaiveConformityScore,0.977027,0.995946,1.122973
4,dermatology (6),0.05,classifier+RAPSConformityScore,0.977027,0.995946,1.106757
...,...,...,...,...,...,...
100,wine (3),0.20,classifier+Naive,0.987374,0.833333,0.833333
101,wine (3),0.20,classifier+NaiveConformityScore,0.987374,0.987374,1.025253
102,wine (3),0.20,classifier+RAPSConformityScore,0.987374,0.987374,1.017677
103,wine (3),0.20,classifier+TopKConformityScore,0.987374,0.987374,1.012626


In [29]:
grouped_df

,dataset,alpha,method,score_acc,coverage_mean,efficiency_mean
0,dermatology (6),0.05,classifier+APSConformityScore,0.977027,0.995946,1.086486
1,dermatology (6),0.05,classifier+LACConformityScore,0.977027,0.997297,1.127027
2,dermatology (6),0.05,classifier+Naive,0.977027,0.970270,0.986486
3,dermatology (6),0.05,classifier+NaiveConformityScore,0.977027,0.995946,1.122973
4,dermatology (6),0.05,classifier+RAPSConformityScore,0.977027,0.995946,1.106757
...,...,...,...,...,...,...
100,wine (3),0.20,classifier+Naive,0.987374,0.833333,0.833333
101,wine (3),0.20,classifier+NaiveConformityScore,0.987374,0.987374,1.025253
102,wine (3),0.20,classifier+RAPSConformityScore,0.987374,0.987374,1.017677
103,wine (3),0.20,classifier+TopKConformityScore,0.987374,0.987374,1.012626
